倒立摆环境实验

In [6]:
import gym
env = gym.make("CartPole-v0")


d:\conda\envs\d2l\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [7]:
env.reset()

(array([-0.04579001,  0.03924654, -0.03609226, -0.04957405], dtype=float32),
 {})

In [8]:
print("env.action_space: ", env.action_space)

env.action_space:  Discrete(2)


In [11]:
#执行一个向左的操作
obj, reward, done,_, info = env.step(1) #1 向右 0向左
print("obj", obj) # 新状态
print("reward", reward) # 指定该动作获取的奖励值
print("done", done) # 回合是否结束
print("info", info) # 额外信息

obj [-0.03169782  0.62620384 -0.05730348 -0.9638166 ]
reward 1.0
done False
info {}


In [12]:
#随机获取一个动作
action = env.action_space.sample()
print(action)

0


In [14]:
env.reset()
done = True

for step in range(50):
    if done:
        state = env.reset()
    action = env.action_space.sample()
    state, reward, done,_, info = env.step(action)

env.close()


调包的整完了，手搓一个试试

In [17]:
import gym 
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [18]:
learning_rate = 0.0005 
gamma = 0.98 
lmbda = 0.95
eps_clip = 0.1 
K_epoch = 3 
T_horizon = 20 

直接开始搓马里奥

In [2]:
import gym_super_mario_bros
from gym.spaces import Box
from gym import Wrapper
from nes_py.wrappers import JoypadSpace#BinarySpaceToDiscreteSpaceEnv
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY
import cv2
import numpy as np
import subprocess as sp

In [ ]:
def process_frame(frame):
    if frame is not None:
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) #图像转换
        
        frame = cv2.resize(frame, (84, 84))[None, :, :] / 255. #裁剪合适大小，并归一化

        return frame
    else:
        return np.zeros((1,84, 84))

In [ ]:

class CustomSkipFrame(Wrapper):
    def __init__(self, env, skip=4):
        super(CustomSkipFrame, self).__init__(env)
        self.observation_space = Box(low=0, high=255, shape=(4, 84, 84))
        self.skip = skip

    def step(self, action):
        total_reward = 0
        states = []
        state, reward, done, info = self.env.step(action)
        for i in range(self.skip):
            if not done:
                state, reward, done, info = self.env.step(action)
                total_reward += reward
                states.append(state)
            else:
                states.append(state)
        states = np.concatenate(states, 0)[None, :, :, :]
        return states.astype(np.float32), reward, done, info

    def reset(self):
        state = self.env.reset()
        states = np.concatenate([state for _ in range(self.skip)], 0)[None, :, :, :]
        return states.astype(np.float32)



In [ ]:
class CustomReward(Wrapper):
    def __init__(self, env=None):
        super(CustomReward, self).__init__(env)
        self.observation_space = Box(low=0, high=255, shape=(1, 84, 84))
        self.curr_score = 0
        self.pre_x = 38 
    def step(self, action):
        state, reward, done, info = self.env.step(action) 
        state = process_frame(state)
        reward += (info["score"] - self.curr_score) / 50.
        
        reward += (info["x_pos"] - self.pre_x ) * 10.
        self.pre_x = info["x_pos"]
        # self.pre_x = info["x_pos"]
        self.curr_score = info["score"]
        if done:
            if info["flag_get"]:
                reward += 5000
            else:
                reward -= 50
        return state, reward/10. , done, info

    def reset(self):
        self.curr_score = 0
        return process_frame(self.env.reset())

In [ ]:
#至此，我们完成了超级玛丽环境的自定义，封装如下：
def create_train_env(world, stage, action_type, output_path=None):
    env = gym_super_mario_bros.make("SuperMarioBros-{}-{}-v0".format(world, stage))
    if action_type == "right":
        actions = RIGHT_ONLY
    elif action_type == "simple":
        actions = SIMPLE_MOVEMENT
    else:
        actions = COMPLEX_MOVEMENT
    env = JoypadSpace(env, actions)
    env = CustomReward(env)
    env = CustomSkipFrame(env)
    return env, env.observation_space.shape[0], len(actions)

In [1]:
import numpy as np
import scipy.signal
from gym.spaces import Box, Discrete

import torch
import torch.nn as nn
from torch.distributions.normal import Normal
from torch.distributions.categorical import Categorical
import torch.nn.functional as F

In [ ]:
def combined_shape(length,shape=None):
    if shape is None :
        return (length,)
    return (length,shape) if np.isscalar(shape) else (length,*shape)
    # np.isscalar 判断对象是否为标量值

In [ ]:
class cnn_model(nn.Module):
    def __init__(self,num_inputs,num_out,activation = nn.ReLU):
        super(cnn_model,self).__init__()
        self.conv1 = nn.Conv2d(num_inputs,32,3,stride = 2 ,padding =1)
        self.conv2 = nn.Conv2d(32,32,3,stride = 2 ,padding =1)
        self.conv3 = nn.Conv2d(32,32,3,stride = 2 ,padding =1)
        self.conv4 = nn.Conv2d(32,32,3,stride = 2 ,padding =1)
        self.lstm = nn.Linear(326*6,512)
        self.fc_out = nn.Linear(512,num_out)
        self._initialize_weights() # 初始化权重
    def _initialize_weights(self):
        for module in self.modules: # 以迭代器的方式返回此前声明的所有layer
            if isinstance(module,nn.Conv2d) or isinstance(module,nn.Linear): #isinstance,判断参数一是否是参数二的格式
                nn.init.xavier_uniform_(module.weight) # 使用均匀分布 用值填充输入张量
                nn.init.constant_(module.bis,0)  # 使用 输入值来为bis填入张量
            elif isinstance(module,nn.LSTMCell):
                nn.init.constant_(module.bis_ih,0) 
                nn.init.constant_(module.bis_hh,0) 
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(x.size(0),-1)
        x = F.relu(self.lstm(x))
        out = self.fc_out(x)
        return out 

In [ ]:
def count_var(module):#计算参数总数
    return sum([np.prod(p.shape) for p in module.parameters()]) 
def discount_cumsum(x,discount):
    return scipy.signal.lfilter([1],[1,float(-discount)],x[::-1],axis = 0)[::-1]

In [ ]:
class 

In [1]:
from spinup.utils.logx import EpochLogger

In [2]:
import scipy.signal

d:\conda\envs\d2l\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
